In [1]:
import pandas as pd
import numpy as np
import scipy.optimize
import warnings


from pyrb import EqualRiskContribution, RiskBudgeting, RiskBudgetAllocation

In [2]:
def calc_weights(cov, x0=None, options=None, scale_factor=10000,
                 pcr_tolerance=0.001, ignore_objective=False):
    """
    Calculate the weights associated with the equal risk contribution
    portfolio. Refer to "On the Properties of Equally-Weighted Risk
    Contributions Portfolios" by Maillard, Roncalli, and  Teiletche for
    definitions.
    Parameters
    ----------
    cov: numpy.ndarray
        (N, N) covariance matrix of assets, must be positive definite
    x0: numpy.ndarray
        (N,) initial solution guess. If None is given uses inverse of standard
        deviation regularized to be between 0 and 1.
    options: dictionary
        A dictionary of solver options. See scipy.optimize.minimize.
    scale_factor: float
        Number to scale the optimization function by, can be helpful for
        convergence
    pcr_tolerance: float
        The max allowable tolerance for differences in the PCR coming from
        different assets in decimal terms, e.g. 1% would be 0.01
    ignore_objective: False
        Provided the max difference in PCR satifies pcr_tolerance, ignore
        whether the objective function has converged. See Notes below.
    Returns
    -------
    w: numpy.ndarray
        (N,) array of asset weights
    Notes:
    ------
    The objective function from the paper embodies but is not exactly
    the same as the desired result, which is to have equal risk contributions
    in terms of PCR  for each asset. As a result, there are scenarios where the
    maxiter will be exceeded (i.e. non convergence) when in fact the goal of
    having equal risk contributions within some acceptable tolerance has been
    achieved. In these scenaries playing around with 'ftol' and 'maxiter' in
    'options' and 'scale_factor' is helpful. The objective function can also
    be ignored using ignore_objective=True, meaning the weights will be
    returned provided the max PCR tolerance is satiesfied even if the objective
    has not converged. See https://github.com/matthewgilbert/erc/issues/1
    """

    # check matrix is PD
    np.linalg.cholesky(cov)

    if not options:
        options = {'ftol': 1e-20, 'maxiter': 800}

    def fun(x):
        # these are non normalized risk contributions, i.e. not regularized
        # by total risk, seems to help numerically
        risk_contributions = x.dot(cov) * x
        a = np.reshape(risk_contributions, (len(risk_contributions), 1))
        # broadcasts so you get pairwise differences in risk contributions
        risk_diffs = a - a.transpose()
        sum_risk_diffs_squared = np.sum(np.square(np.ravel(risk_diffs)))
        # https://stackoverflow.com/a/36685019/1451311
        return sum_risk_diffs_squared / scale_factor

    N = cov.shape[0]
    if x0 is None:
        x0 = 1 / np.sqrt(np.diag(cov))
        x0 = x0 / x0.sum()

    bounds = [(0, 1) for i in range(N)]
    constraints = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}
    res = scipy.optimize.minimize(fun, x0, method='SLSQP', bounds=bounds,
                                  constraints=constraints,
                                  options=options)
    weights = res.x
    risk_squared = weights.dot(cov).dot(weights)
    pcrs = weights.dot(cov) * weights / risk_squared
    pcrs = np.reshape(pcrs, (len(pcrs), 1))
    pcr_max_diff = np.max(np.abs(pcrs - pcrs.transpose()))
    if not res.success:
        if ignore_objective and (pcr_max_diff < pcr_tolerance):
            return weights
        else:
            msg = ("Max difference in percentage contribution to risk "
                   "in decimals is {0:.2E}, "
                   "tolerance is {1:.2E}".format(pcr_max_diff, pcr_tolerance))
            warnings.warn(msg)
            raise RuntimeError(res)
    if pcr_max_diff > pcr_tolerance:
        raise RuntimeError("Max difference in percentage contribution to risk "
                           "in decimals is %s which exceeds tolerance of %s." %
                           (pcr_max_diff, pcr_tolerance))

    return weights

In [3]:
cov = np.array([[2, 0], [0, 1]])
res = calc_weights(cov)
pcr = res.dot(cov) * res / (res.dot(cov).dot(res))

In [4]:
print('Actual: {}'.format(pcr))
print('Predicted: {}'.format(np.ones(2)/2))

Actual: [0.5 0.5]
Predicted: [0.5 0.5]


In [5]:
np.random.seed(42)
# this will be PSD in general, for this seed PD was checked manually
A = np.random.randn(10, 10)
cov = A.dot(A.transpose())
res = calc_weights(cov)
pcr = res.dot(cov) * res / (res.dot(cov).dot(res))
pcr_exp = np.ones(len(A)) / len(A)


In [8]:
pcr

array([0.09999999, 0.09999956, 0.10000052, 0.09999991, 0.0999998 ,
       0.09999978, 0.10000027, 0.10000001, 0.0999999 , 0.10000026])

In [6]:
print('Actual: {}'.format(pcr))
print('Predicted: {}'.format(np.ones(len(A))/len(A)))

Actual: [0.09999999 0.09999956 0.10000052 0.09999991 0.0999998  0.09999978
 0.10000027 0.10000001 0.0999999  0.10000026]
Predicted: [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]


In [7]:
res

array([0.12800433, 0.07478389, 0.02192582, 0.07737148, 0.07021499,
       0.30410349, 0.09122203, 0.03795839, 0.12980398, 0.06461159])

In [46]:
# try with pyrb

In [52]:
# set risk budget
budgets = [0.1,0.1,0.1,0.2,0.2,0.05,0.05,0.05,0.05,0.1]

In [49]:
ERC = EqualRiskContribution(cov)
ERC.solve()

In [50]:
ERC.x

array([0.12800218, 0.07478356, 0.02192644, 0.07737002, 0.07021301,
       0.30410379, 0.09122209, 0.0379593 , 0.12980594, 0.06461366])

In [51]:
ERC.get_risk_contributions()

array([0.09999151, 0.10000151, 0.10000332, 0.09999667, 0.09999746,
       0.09999869, 0.10000146, 0.10000313, 0.10000311, 0.10000313])

In [54]:
RB = RiskBudgeting(cov,budgets)
RB.solve()

In [55]:
RB.x

array([0.14139763, 0.06993647, 0.02031686, 0.09673815, 0.10609524,
       0.30244623, 0.08329459, 0.02354623, 0.10975172, 0.04647688])

In [56]:
RB.get_risk_contributions()

array([0.0999918 , 0.10000238, 0.10000359, 0.19999885, 0.19999855,
       0.0499976 , 0.05000036, 0.05000174, 0.05000173, 0.10000342])

In [10]:
cov = np.array([[ 1.18172492e-05,  1.02638721e-05,  9.51320757e-06,
         1.03466520e-05,  8.87293447e-06,  2.90059111e-06,
         3.40475492e-06,  8.40904264e-06,  4.62700964e-06,
         6.30296648e-06,  6.02767397e-06,  8.97473251e-06,
         5.45963932e-06,  1.31006015e-05, -8.17297169e-06,
         5.46118683e-07],
       [ 1.02638721e-05,  2.47042278e-05,  4.96813675e-06,
         1.98193133e-05,  9.47533265e-06, -9.93309829e-07,
         6.38099624e-06,  7.82383643e-06,  4.07312122e-06,
         5.00038996e-06,  8.38956448e-06,  1.55775458e-05,
         1.04467373e-05,  1.30992892e-05, -1.31381537e-05,
        -6.88067959e-08],
       [ 9.51320757e-06,  4.96813675e-06,  3.00289488e-04,
         1.55756263e-05,  2.22765195e-04,  2.08005293e-04,
         6.67102033e-05,  1.60641953e-06,  8.81394506e-05,
         2.40589483e-04,  1.79006949e-04,  1.63751220e-04,
         1.69183824e-04,  8.01956747e-05, -4.70681473e-04,
        -7.87545559e-06],
       [ 1.03466520e-05,  1.98193133e-05,  1.55756263e-05,
         8.29232062e-05,  1.96658502e-05,  8.22902539e-06,
         1.39743100e-05,  1.18813550e-05,  7.31503383e-06,
         6.70499920e-06,  1.53651426e-05,  2.11974864e-05,
         1.95576187e-05,  1.44144769e-05, -2.06931223e-05,
         5.84761929e-06],
       [ 8.87293447e-06,  9.47533265e-06,  2.22765195e-04,
         1.96658502e-05,  2.20230546e-04,  1.91107359e-04,
         7.61644303e-05,  3.69853861e-06,  7.53227996e-05,
         2.15437300e-04,  1.81033160e-04,  1.55418630e-04,
         1.73661912e-04,  7.77463337e-05, -4.61586645e-04,
        -5.68000120e-06],
       [ 2.90059111e-06, -9.93309829e-07,  2.08005293e-04,
         8.22902539e-06,  1.91107359e-04,  2.22051737e-04,
         7.79516917e-05, -4.58920253e-06,  7.44737390e-05,
         2.32075853e-04,  1.83210592e-04,  1.44393234e-04,
         1.87761372e-04,  6.69760865e-05, -5.67142926e-04,
        -5.58658601e-06],
       [ 3.40475492e-06,  6.38099624e-06,  6.67102033e-05,
         1.39743100e-05,  7.61644303e-05,  7.79516917e-05,
         1.17404379e-04,  4.99124572e-06,  2.91989905e-05,
         8.98400771e-05,  7.80953780e-05,  6.85677360e-05,
         8.15845099e-05,  3.79857733e-05, -2.62784130e-04,
         5.30739683e-06],
       [ 8.40904264e-06,  7.82383643e-06,  1.60641953e-06,
         1.18813550e-05,  3.69853861e-06, -4.58920253e-06,
         4.99124572e-06,  1.23766466e-05,  6.06263803e-07,
        -1.17038130e-06,  6.73600118e-07,  4.17317128e-06,
         1.90074460e-07,  1.16202264e-05, -1.92178934e-06,
         1.28469153e-06],
       [ 4.62700964e-06,  4.07312122e-06,  8.81394506e-05,
         7.31503383e-06,  7.53227996e-05,  7.44737390e-05,
         2.91989905e-05,  6.06263803e-07,  8.49772946e-05,
         8.50394078e-05,  6.63082172e-05,  5.77687090e-05,
         6.52970093e-05,  3.17574152e-05, -1.67489894e-04,
        -3.83904569e-06],
       [ 6.30296648e-06,  5.00038996e-06,  2.40589483e-04,
         6.70499920e-06,  2.15437300e-04,  2.32075853e-04,
         8.98400771e-05, -1.17038130e-06,  8.50394078e-05,
         2.78689303e-04,  2.03322007e-04,  1.67027149e-04,
         2.01166648e-04,  8.08468113e-05, -6.08656191e-04,
        -7.61676737e-06],
       [ 6.02767397e-06,  8.38956448e-06,  1.79006949e-04,
         1.53651426e-05,  1.81033160e-04,  1.83210592e-04,
         7.80953780e-05,  6.73600118e-07,  6.63082172e-05,
         2.03322007e-04,  1.82252367e-04,  1.48034827e-04,
         1.81161783e-04,  6.84656934e-05, -4.94712852e-04,
        -5.38208243e-06],
       [ 8.97473251e-06,  1.55775458e-05,  1.63751220e-04,
         2.11974864e-05,  1.55418630e-04,  1.44393234e-04,
         6.85677360e-05,  4.17317128e-06,  5.77687090e-05,
         1.67027149e-04,  1.48034827e-04,  1.53905166e-04,
         1.56289114e-04,  6.50131399e-05, -3.96532141e-04,
        -5.56830874e-06],
       [ 5.45963932e-06,  1.04467373e-05,  1.69183824e-04,
         1.95576187e-05,  1.73661912e-04,  1.87761372e-04,
         8.15845099e-05,  1.90074460e-07,  6.52970093e-05,
         2.01166648e-04,  1.81161783e-04,  1.56289114e-04,
         2.28884146e-04,  7.14533927e-05, -5.26580364e-04,
        -6.10826509e-06],
       [ 1.31006015e-05,  1.30992892e-05,  8.01956747e-05,
         1.44144769e-05,  7.77463337e-05,  6.69760865e-05,
         3.79857733e-05,  1.16202264e-05,  3.17574152e-05,
         8.08468113e-05,  6.84656934e-05,  6.50131399e-05,
         7.14533927e-05,  5.27426342e-05, -1.78980735e-04,
        -2.24946360e-06],
       [-8.17297169e-06, -1.31381537e-05, -4.70681473e-04,
        -2.06931223e-05, -4.61586645e-04, -5.67142926e-04,
        -2.62784130e-04, -1.92178934e-06, -1.67489894e-04,
        -6.08656191e-04, -4.94712852e-04, -3.96532141e-04,
        -5.26580364e-04, -1.78980735e-04,  2.44791520e-03,
         5.78398536e-06],
       [ 5.46118683e-07, -6.88067959e-08, -7.87545559e-06,
         5.84761929e-06, -5.68000120e-06, -5.58658601e-06,
         5.30739683e-06,  1.28469153e-06, -3.83904569e-06,
        -7.61676737e-06, -5.38208243e-06, -5.56830874e-06,
        -6.10826509e-06, -2.24946360e-06,  5.78398536e-06,
         1.63382854e-05]])

In [11]:
cov.shape

(16, 16)

In [23]:
res = calc_weights(cov,scale_factor=.001,pcr_tolerance=0.1)
pcr = res.dot(cov) * res / (res.dot(cov).dot(res))

In [27]:
res

array([1.39992864e-01, 1.06187701e-01, 1.95756503e-02, 5.45265253e-02,
       2.01581385e-02, 2.27458443e-02, 3.65297833e-02, 1.65322470e-01,
       4.85125388e-02, 1.93009764e-02, 2.24942168e-02, 2.40386179e-02,
       2.17587346e-02, 3.94169131e-02, 3.27179711e-18, 2.59439026e-01])